<a href="https://colab.research.google.com/github/polydeuces32/Blockchain-Development-Resources/blob/main/weather_perdiction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install pandas matplotlib statsmodels

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [ ]:
# Sample weather dataset URL (Replace with your dataset URL or file path)
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'

# Load dataset
df = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Display the first few rows of the dataset
df.head()


In [ ]:
# Plot the time series data
plt.figure(figsize=(10, 6))
plt.plot(df, label='Daily Minimum Temperature')
plt.title('Daily Minimum Temperature Time Series')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()


In [ ]:
# Differencing the time series
df_diff = df.diff().dropna()

# Check stationarity again
result = adfuller(df_diff['Temp'])
print('ADF Statistic after differencing:', result[0])
print('p-value after differencing:', result[1])

# Plot the differenced series
plt.figure(figsize=(10, 6))
plt.plot(df_diff, label='Differenced Series')
plt.title('Differenced Series')
plt.xlabel('Date')
plt.ylabel('Differenced Temperature')
plt.legend()
plt.show()


In [ ]:
# Plot ACF and PACF
plot_acf(df_diff, lags=30)
plot_pacf(df_diff, lags=30)
plt.show()


In [ ]:
# Fit ARIMA/SARIMA model
model = SARIMAX(df, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
model_fit = model.fit(disp=False)
print(model_fit.summary())


In [ ]:
# Make predictions
predictions = model_fit.predict(start=len(df), end=len(df) + 30, dynamic=False)

# Plot predictions
plt.figure(figsize=(10, 6))
plt.plot(df, label='Observed')
plt.plot(predictions, label='Forecast', color='red')
plt.title('Weather Forecast')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()


In [ ]:
!pip install requests


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Replace with your actual API key from OpenWeatherMap
api_key = '82ca01b918d212c5df3bc7bcecdaec4a'

# Define the city and the date range
city = 'New York City'
start_date = datetime(2024, 6, 10)  # Manually specify the start date
end_date = datetime(2024, 6, 17)    # Manually specify the end date

# Calculate the number of days in the range
num_days = (end_date - start_date).days

# Create a list to hold the weather data
weather_data = []

# Loop to simulate fetching weather data for multiple days
for i in range(num_days + 1):
    date = start_date + timedelta(days=i)
    formatted_date = date.strftime('%Y-%m-%d')

    # Fetch the current weather data
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric'
    response = requests.get(url)
    data = response.json()

    # Check if 'main' key exists in the response data
    if 'main' in data:
        # Extract the current temperature
        current_temp = data['main']['temp_min']

        # Append the data to the list
        weather_data.append({
            'Date': formatted_date,
            'MinTemp': current_temp
        })
    else:
        print(f"No weather data available for {city} on {formatted_date}")

# Convert the list to a DataFrame
df = pd.DataFrame(weather_data)

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(df['Date'], df['MinTemp'], marker='o', linestyle='-', color='b')
plt.title(f'Daily Minimum Temperature in {city}')
plt.xlabel('Date')
plt.ylabel('Minimum Temperature (°C)')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()
